email_path.txt consists of path to emails. **Spam emails** 52790 and **ham emails** 39398

In [1]:
import pandas as pd
import os
import mailparser
from nlpia.loaders import get_data
from nltk.tokenize import casual_tokenize
from matplotlib import pyplot as plt
import seaborn
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')
import collections
#pd.set_option('display.width', 1000)
#pd.set_option('display.height', 1000)
#pd.set_option('max_colwidth', 1000)

In [2]:
def getting_emails():
    email_list = []
    f1 = open('email_path.txt', 'r')
    count = 0
    for line in f1:
        count += 1
        l1= line.split(',')
        path_mail = l1[0]
        isspam = l1[1]
        if (count<=2600):
            mail = mailparser.parse_from_file(path_mail)
            email_list.append({'path': path_mail, 'Date':mail.date, 'Body': mail.body, \
                          'From':mail.from_, 'message_id': mail.message_id, 'subject': mail.subject,\
                          'To': mail.to, 'isspam': isspam})
        else:
            break
    email_list_df = pd.DataFrame(email_list) 
    email_list_df.replace(to_replace= 'spam \n', value= 0, regex=True, inplace= True)
    email_list_df.replace(to_replace= 'ham \n', value= 1, regex=True, inplace= True)
    #email_list_df.head()
    return(email_list_df)

In [4]:
# create test-train set
def spam_test_train_set():
    email_list_df = getting_emails()
    ham_email_list = email_list_df[(email_list_df.isspam == 1)].iloc[:1000]
    spam_email_list = email_list_df[(email_list_df.isspam == 0)].iloc[:1000]
    train_email = pd.concat([ham_email_list.iloc[:800], spam_email_list.iloc[:800]])
    test_email = pd.concat([ham_email_list.iloc[800:], spam_email_list.iloc[800:]])
    return(train_email, test_email)

### efficient memory use

In [5]:
def iter_strs(strs):
    for s in strs:
        tokens = s.split()
        counts = collections.Counter(tokens)
        yield counts

In [6]:
def confusion(predictions, actuals):
    actuals=actuals.values[:,0] if isinstance(actuals,pd.DataFrame) else actuals
    
    true_pos= (predictions==1) & (actuals==1)
    true_pos.sum()
    true_neg= (predictions==0) & (actuals==0)
    true_neg.sum()
    false_pos= (predictions==1) & (actuals==0)
    false_pos.sum()
    false_neg= (predictions==0) & (actuals==1)
    false_neg.sum()
    
    prec=true_pos.sum()/(true_pos.sum()+false_pos.sum())
    accur=(true_pos.sum()+true_neg.sum())/(true_pos.sum()+false_pos.sum()+ \
                                           true_neg.sum()+ false_neg.sum())
    recall = true_pos.sum()/(true_pos.sum()+false_neg.sum())
    F1=2*(prec*recall/(prec+recall))
 
    return(true_pos.sum(), false_pos.sum(),false_neg.sum(),true_neg.sum(), accur,recall, prec, F1)